In [100]:
import numpy as np
A = [[1, 1, 2, 2], [2, 1, 1, 2], [2, 2, 1, 1], [1, 2, 2, 1]]
B = np.array([[1, 1, 2, 2], [2, 2, 1, 1], [2, 2, 1, 1], [1, 2, 2, 1]], dtype=np.int8)
C = np.array([
            [0, 0, 0, 0], 
            [0, 1, 0, 1], 
            [0, 0, 2, 0], 
            [0, 1, 0, 0]], dtype=np.int8)
D = np.array([
            [1, 1, 0, 0], 
            [0, 1, 1, 0], 
            [0, 0, 1, 1], 
            [1, 0, 0, 1]], dtype=np.int8)
E = np.array([  # Unique 
            [1, 2, 2, 1], 
            [2, 1, 1, 2], 
            [2, 1, 0, 0], 
            [1, 2, 0, 0]], dtype=np.int8)


## Solver

In [121]:
from duko import Duko
import numpy as np
duko = Duko(render=False, board=D)
board = duko.board.astype(np.int8)
N = len(board)
C

array([[0, 0, 0, 0],
       [0, 1, 0, 1],
       [0, 0, 2, 0],
       [0, 1, 0, 0]], dtype=int8)

In [158]:
# one hot encode the board
# ONE, TWO = 
# find indices of 1s and 2s
BLUE, RED = np.zeros((2, 4, 4))
BLUE[np.where(board == 1)] = RED[np.where(board == 2)] = 1
DIGITS = set(range(len(board)))
done_cols, done_rows = set(), set()  # Always start empty?
BOARD = C.copy()
FLIP = np.array([0, 2, 1])

def equal_count(grid, done: set):
    """Checks for equal count of 1s and 2s.
    param grid: 2D array
    param done: set of rows or cols that are already done
    Returns True if any changes are made."""
    changed = False

    for r in (DIGITS - done):
        row = grid[r]
        if np.sum(row == 1) == N // 2:
            grid[r, np.where(row == 0)[0]] = 2
            done.add(r)
            changed = True

        if np.sum(row == 2) == N // 2:
            grid[r, np.where(row == 0)[0]] = 1
            done.add(r)
            changed = True

    return changed

def subsequent(grid, done: set):
    """Checks for subsequent 1s and 2s in rows and cols.
    Returns True if any changes are made."""
    changed = False

    for r, row in enumerate(grid):
        for i in range(len(row)-1):
            if row[i] == row[i+1] != 0:
                if i < N-2 and not grid[r, i+2]: # if next is empty
                    grid[r, i+2] = FLIP[row[i]]
                    changed = True

                if i > 0 and not grid[r, i-1]: # if prev is empty
                    grid[r, i-1] = FLIP[row[i]]
                    changed = True

    if changed: # TODO: check funcionatliy
        done.update(np.where(np.all(board != 0, axis=1))[0])

    return changed

def unique(grid, done: set):
    """Checks for unique rows and cols.
    Returns True if any changes are made."""
    changed = False

    # Find rows containing exacty two gaps
    gapped_rows_indx = np.where(np.sum(grid == 0, axis=1) == 2)[0]  # [2 3]
    candidates = grid[gapped_rows_indx]  # [[2 1 0 0], [1 0 0 2]]
    keks = grid[list(done)]  # [[2 1 1 2]]

    # Compare each candidate with other complete rows
    for i, candy in enumerate(candidates):
        # Find index of colored tiles
        idx = np.where(candy != 0)[0]  # [0 1]
        for kek in keks:
            if np.array_equal(kek[idx], candy[idx]):
                # Replace the two gaps with complementary colors
                gaps = np.where(candy == 0)[0]  # [2, 3]
                grid[gapped_rows_indx[i], gaps] = FLIP[kek[gaps]]

                done.add(gapped_rows_indx[i])
                changed = True
    
    return changed

# print(equal_count(BOARD, done_rows))
# print(BOARD)
# print(done_rows)
# print(subsequent(BOARD.T, done_cols))
# print(BOARD)
# print(done_cols)
for i in range(10):
    print(equal_count(BOARD, done_rows))
# array([[0, 0, 0, 0],
#        [2, 1, 2, 1],
#        [0, 0, 2, 0],
#        [0, 1, 0, 0]], dtype=int8)

# array([[0, 2, 1, 0],
#        [2, 1, 2, 1],
#        [0, 2, 2, 0],
#        [0, 1, 1, 0]], dtype=int8)


True
False
False
False
False
False
False
False
False
False


In [156]:
        pos = (self.SIZE // 2, self.SIZE * 3 // 5)
        self.outline_text(congratz, "Complete!", pos)
        pos = (self.SIZE // 2, self.SIZE * 5 // 8)
        self.outline_text(congratz, "Press space to continue", pos, self.font_med)

    def outline_text(self, SURFACE: pg.Surface, text, pos, 
                     font=font_big, color=WHITE, ocolor=BLACK, opx=2):
        """Draw text with outline"""
        surface = font.render(text, True, color).convert_alpha()
        w, h = surface.get_width(), font.get_height()

        surf_outline = pg.Surface((w + 2 * opx, h + 2 * opx)).convert_alpha()
        # surf_outline.fill(self.BLACK)

        surf = surf_outline.copy()
        surf_outline.blit(font.render(text, True, ocolor).convert_alpha(), (0, 0))

        for dx, dy in self._circlepoints(opx):
            surf.blit(surf_outline, (dx + opx, dy + opx))

        surf.blit(surface, (opx, opx))
        offset_pos = (pos[0] - w // 2, pos[1] - h // 2)
        SURFACE.blit(surf, offset_pos)

0
1
2
3
4
5
6
7
8
9


In [ ]:
def equal_count0():
    """Checks for equal count of 1s and 2s in rows and cols.
    Returns True if any changes are made."""
    changed = False

    BLUE_cols = np.sum(BLUE, axis=1)  # Summing over cols [0., 2., 0., 1.]
    cols = set(np.where(BLUE_cols == 2)[0]) - done_cols # index of filled # {1}
    for c in cols:
        done_cols.add(c)

        rows = np.where(BLUE[:, c] == 1)[0]  # [1, 3]
        left_over = DIGITS - set(rows) # {0, 2}
        RED[tuple(left_over), c] = 1
        changed = True

    
    BLUE_rows = np.sum(BLUE, axis=0)
    rows = set(np.where(BLUE_rows == 2)[0]) - done_rows
    for r in rows:
        done_rows.add(r)

        cols = np.where(BLUE[r, :] == 1)[0] 
        left_over = DIGITS - set(cols)
        RED[r, tuple(left_over)] = 1
        changed = True
    
    RED_cols = np.sum(RED, axis=0)
    cols = set(np.where(RED_cols == 2)[0]) - done_cols # index of filled # {1}
    for c in cols:
        done_cols.add(c)

        rows = np.where(RED[:, c] == 1)[0]  # [1, 3]
        left_over = DIGITS - set(rows) # {0, 2}
        BLUE[tuple(left_over), c] = 1
        changed = True

    RED_rows = np.sum(RED, axis=1)
    rows = set(np.where(RED_rows == 2)[0]) - done_rows
    for r in rows:
        done_rows.add(r)

        cols = np.where(RED[r, :] == 1)[0] 
        left_over = DIGITS - set(cols)
        BLUE[r, tuple(left_over)] = 1
        changed = True

    return changed


In [129]:
# A = np.array([0, 1, 2, 3])
# # Determine number of zeros
# A[[1, 3]] = [-1, -3]
# A
A = np.array([2, 1, 0, 1])
# get opposite of A

# blue_rows
# np.where(board == 1)

C:\Users\Jason\AppData\Local\Temp\ipykernel_14244\1025829497.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  np.sum(A == np.array([1, 0]))


0

In [20]:
# SAVE AND LOAD configirations

# save = {str(i) : a for i, a in enumerate(configs)}  # Give them names 
configs = [[1, 0], A, B]  # Or just use a list
np.savez("duko_configs.npz", *configs)  # Save them

loaded = np.load("duko_configs.npz")
configs = loaded.values()
for kek in configs:
    print(kek)

[1 0]
[[1 1 2 2]
 [2 1 1 2]
 [2 2 1 1]
 [1 2 2 1]]
[[1 1 2 2]
 [2 2 1 1]
 [2 2 1 1]
 [1 2 2 1]]


In [6]:
# load the config from disk
config = np.loadtxt("duko_config.txt", dtype=np.int8)

[[ 9 11  9  7]
 [ 9 10  9  8]
 [ 9 10  9  8]
 [10 11  8  7]]


In [54]:
dim = A.shape[0]
for i in range(dim):
    for j in range(i, dim):
        if i != j:
            # print(i, j)
            # print(A[i])
            # print(A[j])
            # print((A[i] == A[j]))
            print((A[i] == A[j]).all())

            # print()
            print((A[:, i] == A[:, j]).all())
            # if np.all(A[i] == A[j]) \
            # or np.all(A[:, i] == A[:, j]):
            #     print("False")
print(A)

False
False
False
False
False
False
True
False
False
False
False
False
[[1 1 2 2]
 [2 2 1 1]
 [2 2 1 1]
 [1 2 2 1]]


In [15]:
dim = 4
[(x, y, a) for x in range(dim) for y in range(dim) for a in range(1, 3)]

[(0, 0, 1),
 (0, 0, 2),
 (0, 1, 1),
 (0, 1, 2),
 (0, 2, 1),
 (0, 2, 2),
 (0, 3, 1),
 (0, 3, 2),
 (1, 0, 1),
 (1, 0, 2),
 (1, 1, 1),
 (1, 1, 2),
 (1, 2, 1),
 (1, 2, 2),
 (1, 3, 1),
 (1, 3, 2),
 (2, 0, 1),
 (2, 0, 2),
 (2, 1, 1),
 (2, 1, 2),
 (2, 2, 1),
 (2, 2, 2),
 (2, 3, 1),
 (2, 3, 2),
 (3, 0, 1),
 (3, 0, 2),
 (3, 1, 1),
 (3, 1, 2),
 (3, 2, 1),
 (3, 2, 2),
 (3, 3, 1),
 (3, 3, 2)]

In [13]:
A = np.array([[1, 1, 0, 1], [1, 1, 0, 1], [1, 1, 0, 1], [1, 1, 0, 1]], dtype=np.int8)
goal = np.ones(4, dtype=np.int8) * 3
row_sum = np.sum(A, axis=1)
# CHeck if all entrees in row_sum are equal to goal
print((row_sum == goal).all())

False
